# DPF Visualization Workshop

This notebook demonstrates **PyDPF-Core visualization capabilities** using the DPF Plotter. Visualization is essential for understanding simulation results and communicating findings effectively.

## What You'll Learn

- Basic field plotting
- Mesh visualization
- Deformed shape display
- Contour plots with custom settings
- Animation of transient results
- Multiple plot comparisons
- Custom visualization settings

## Setup: Load Model and Extract Data

In [1]:
from ansys.dpf import core as dpf
from ansys.dpf.core import examples
import numpy as np

# Load a transient analysis result file
model = dpf.Model(examples.download_transient_result())

print("Model loaded successfully!")
print(f"Result file: {model.metadata.data_sources.result_key}")
print(f"Number of time steps: {len(model.metadata.time_freq_support.time_frequencies)}")
print(f"Analysis type: {model.metadata.result_info.analysis_type}")

Model loaded successfully!
Result file: rst
Number of time steps: 35
Analysis type: static


## Basic Mesh Visualization

### Example 1: Plot the Mesh

The simplest visualization is to display the mesh structure.

In [2]:
# Get the mesh
mesh = model.metadata.meshed_region

# Plot the mesh
mesh.plot()

print(f"Mesh information:")
print(f"  Number of nodes: {mesh.nodes.n_nodes}")
print(f"  Number of elements: {mesh.elements.n_elements}")

Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x216529e1ff0_0&reconnect=auto" class="pyvis…

Mesh information:
  Number of nodes: 3820
  Number of elements: 789


### Example 2: Plot with Custom Settings

In [ ]:
# Plot mesh with different options
mesh.plot(
    show_edges=False,
    opacity=0.1,
)

Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x21614f6b220_3&reconnect=auto" class="pyvis…

(None, <pyvista.plotting.plotter.Plotter at 0x21614f6b220>)

## Field Visualization

### Example 3: Plot Displacement Field

Visualize scalar or vector field data on the mesh.

In [6]:
# Get displacement at a specific time step
displacement = model.results.displacement()
displacement.inputs.time_scoping([30])
disp_fc = displacement.outputs.fields_container()
disp_field = disp_fc[0]

# Plot the displacement magnitude
disp_field.plot()

print(f"Displacement field:")
print(f"  Location: {disp_field.location}")
print(f"  Unit: {disp_field.unit}")
print(f"  Data shape: {disp_field.data.shape}")

Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x216265b42e0_4&reconnect=auto" class="pyvis…

Displacement field:
  Location: Nodal
  Unit: m
  Data shape: (3820, 3)


### Example 4: Plot Displacement Magnitude with Operator Chain

In [8]:
# Create operator chain to get displacement magnitude
disp_op = dpf.operators.result.displacement()
disp_op.inputs.data_sources(model.metadata.data_sources)
disp_op.inputs.time_scoping([30])

# Compute magnitude
norm_op = dpf.operators.math.norm_fc()
norm_op.inputs.fields_container(disp_op.outputs.fields_container)

# Get the magnitude field
disp_mag_field = norm_op.outputs.fields_container()[0]
disp_mag_field.meshed_region = mesh
# Plot
disp_mag_field.plot(title="Displacement Magnitude")

print(f"Max displacement magnitude: {np.max(disp_mag_field.data):.6f} {disp_mag_field.unit}")

Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x216265b43d0_6&reconnect=auto" class="pyvis…

Max displacement magnitude: 0.041883 m


## Deformed Shape Visualization

### Example 5: Plot Deformed Shape

Show how the structure deforms under loading.

In [ ]:
# Plot the deformed mesh
disp_field.plot(
    deform_by=disp_field,
    scale_factor=1.0,
    title="Deformed Shape (Scale Factor = 1.0)"
)

Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x2163c459fc0_8&reconnect=auto" class="pyvis…

(None, <pyvista.plotting.plotter.Plotter at 0x2163c459fc0>)

### Example 6: Exaggerated Deformation for Better Visibility

In [11]:
# Plot with exaggerated deformation
disp_field.plot(
    deform_by=disp_field,
    scale_factor=50.0,
    title="Deformed Shape (Scale Factor = 50.0)"
)

print("Note: Large scale factor helps visualize small deformations")

Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x2163c4904f0_9&reconnect=auto" class="pyvis…

Note: Large scale factor helps visualize small deformations


## Stress Visualization

### Example 7: Plot Von Mises Stress

In [12]:
# Get stress and compute Von Mises
stress_op = dpf.operators.result.stress()
stress_op.inputs.data_sources(model.metadata.data_sources)
stress_op.inputs.time_scoping([30])

von_mises_op = dpf.operators.invariant.von_mises_eqv_fc()
von_mises_op.inputs.fields_container(stress_op.outputs.fields_container)

# Convert to nodal for better visualization
to_nodal_op = dpf.operators.averaging.to_nodal_fc()
to_nodal_op.inputs.fields_container(von_mises_op.outputs.fields_container)

vm_field = to_nodal_op.outputs.fields_container()[0]

# Plot Von Mises stress
vm_field.plot(title="Von Mises Stress")

print(f"Von Mises stress range:")
print(f"  Min: {np.min(vm_field.data):.2f} {vm_field.unit}")
print(f"  Max: {np.max(vm_field.data):.2f} {vm_field.unit}")

Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x2163c615e10_10&reconnect=auto" class="pyvi…

Von Mises stress range:
  Min: 1705526.56 Pa
  Max: 466093820.45 Pa


### Example 8: Von Mises Stress on Deformed Shape

In [13]:
# Plot stress on deformed geometry
vm_field.plot(
    deform_by=disp_field,
    scale_factor=10.0,
    title="Von Mises Stress on Deformed Shape"
)

Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x2163c6168f0_11&reconnect=auto" class="pyvi…

(None, <pyvista.plotting.plotter.Plotter at 0x2163c6168f0>)

## Component-wise Visualization

### Example 9: Plot Individual Displacement Components

In [14]:
# Extract Z component of displacement
component_op = dpf.operators.logic.component_selector_fc()
component_op.inputs.fields_container(disp_fc)
component_op.inputs.component_number(2)  # Z component

z_disp_field = component_op.outputs.fields_container()[0]

# Plot Z displacement
z_disp_field.plot(
    title="Z-Component of Displacement",
    deform_by=disp_field,
    scale_factor=20.0
)

print(f"Z-displacement range:")
print(f"  Min: {np.min(z_disp_field.data):.6f} {z_disp_field.unit}")
print(f"  Max: {np.max(z_disp_field.data):.6f} {z_disp_field.unit}")

Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x21682471f60_12&reconnect=auto" class="pyvi…

Z-displacement range:
  Min: -0.000221 m
  Max: 0.000296 m


## Multiple Time Steps Comparison

### Example 10: Side-by-Side Comparison

In [16]:
# Get displacement at multiple time steps
disp_multi_op = dpf.operators.result.displacement()
disp_multi_op.inputs.data_sources(model.metadata.data_sources)
disp_multi_op.inputs.time_scoping([1, 15, 30])

# Compute magnitude for all time steps
norm_multi_op = dpf.operators.math.norm_fc()
norm_multi_op.inputs.fields_container(disp_multi_op.outputs.fields_container)

disp_mag_fc = norm_multi_op.outputs.fields_container()

# Plot each time step
print("Plotting displacement magnitude at 3 time steps...\n")
for i, field in enumerate(disp_mag_fc):
    time_step = [1, 15, 30][i]
    field.meshed_region = mesh
    field.plot(title=f"Displacement Magnitude - Time Step {time_step}")
    print(f"Time step {time_step}: Max = {np.max(field.data):.6f} {field.unit}")

Plotting displacement magnitude at 3 time steps...



Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x216824726b0_14&reconnect=auto" class="pyvi…

Time step 1: Max = 0.000000 m


Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x21682800760_15&reconnect=auto" class="pyvi…

Time step 15: Max = 0.053584 m


Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x2163c453040_16&reconnect=auto" class="pyvi…

Time step 30: Max = 0.041883 m


## Using the DPF Plotter Directly

### Example 11: Advanced Plotter Control

In [17]:
# Create a plotter instance for more control
from ansys.dpf.core import plotter as dpf_plotter

# Get Von Mises stress field
stress_op = dpf.operators.result.stress()
stress_op.inputs.data_sources(model.metadata.data_sources)
stress_op.inputs.time_scoping([30])

vm_op = dpf.operators.invariant.von_mises_eqv_fc()
vm_op.inputs.fields_container(stress_op.outputs.fields_container)

to_nodal = dpf.operators.averaging.to_nodal_fc()
to_nodal.inputs.fields_container(vm_op.outputs.fields_container)

vm_stress = to_nodal.outputs.fields_container()[0]

# Get displacement for deformation
disp_op = dpf.operators.result.displacement()
disp_op.inputs.data_sources(model.metadata.data_sources)
disp_op.inputs.time_scoping([30])
displacement_field = disp_op.outputs.fields_container()[0]

# Plot with custom settings
vm_stress.plot(
    deform_by=displacement_field,
    scale_factor=5.0,
    show_edges=False,
    title="Von Mises Stress - Custom Plot"
)

Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x2163c452d10_17&reconnect=auto" class="pyvi…

(None, <pyvista.plotting.plotter.Plotter at 0x2163c452d10>)

## Analyzing Specific Regions

### Example 12: Visualize Specific Node Selection

In [18]:
# Create a scoping for a subset of nodes
# Let's select nodes with high displacement
disp_mag_data = disp_mag_field.data
threshold = np.percentile(disp_mag_data, 90)  # Top 10% displacement

# Find nodes exceeding threshold
high_disp_indices = np.where(disp_mag_data > threshold)[0]
high_disp_node_ids = disp_mag_field.scoping.ids[high_disp_indices]

print(f"Total nodes: {len(disp_mag_field.scoping)}")
print(f"Nodes with displacement > {threshold:.6f} {disp_mag_field.unit}: {len(high_disp_node_ids)}")
print(f"Node IDs (first 10): {high_disp_node_ids[:10]}")

# Create scoping
my_scoping = dpf.Scoping()
my_scoping.ids = high_disp_node_ids
my_scoping.location = dpf.locations.nodal

# Get displacement for these nodes only
disp_scoped_op = dpf.operators.result.displacement()
disp_scoped_op.inputs.data_sources(model.metadata.data_sources)
disp_scoped_op.inputs.time_scoping([30])
disp_scoped_op.inputs.mesh_scoping(my_scoping)

disp_scoped_field = disp_scoped_op.outputs.fields_container()[0]

# Note: Plotting partial mesh requires the full mesh context
# We'll plot the full field but highlight the region of interest
disp_mag_field.plot(title="Displacement - Highlighting High Displacement Region")

Total nodes: 3820
Nodes with displacement > 0.040170 m: 382
Node IDs (first 10): [ 975 3664  981  982 3679 3666  988 3680  989 3694]


Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x2169556f160_18&reconnect=auto" class="pyvi…

(None, <pyvista.plotting.plotter.Plotter at 0x2169556f160>)

## Time History Animation (Conceptual)

### Example 13: Extract Data for Animation

In [19]:
# Get displacement for all time steps
disp_all_op = dpf.operators.result.displacement()
disp_all_op.inputs.data_sources(model.metadata.data_sources)

# Get magnitude
norm_all_op = dpf.operators.math.norm_fc()
norm_all_op.inputs.fields_container(disp_all_op.outputs.fields_container)

all_disp_mag_fc = norm_all_op.outputs.fields_container()

print(f"Total time steps available: {len(all_disp_mag_fc)}")
print("\nDisplacement evolution:")

# Sample every 5th time step for visualization
time_steps_to_show = range(0, len(all_disp_mag_fc), 5)

for i in time_steps_to_show:
    field = all_disp_mag_fc[i]
    max_disp = np.max(field.data)
    print(f"  Time step {i+1:3d}: Max displacement = {max_disp:.6f} {field.unit}")

print("\nNote: You can plot individual time steps or create animations using external tools")

Total time steps available: 1

Displacement evolution:
  Time step   1: Max displacement = 0.037338 m

Note: You can plot individual time steps or create animations using external tools


## Exploring Peak Values Visually

### Example 14: Plot Maximum Over Time

In [21]:
# Find maximum displacement across all time steps
max_over_time_op = dpf.operators.min_max.min_max_fc()
max_over_time_op.inputs.fields_container(all_disp_mag_fc)

max_field = max_over_time_op.outputs.field_max()
min_field = max_over_time_op.outputs.field_min()

print("Displacement extremes over entire analysis:")
print(f"  Maximum: {max_field.data[0]:.6f} {max_field.unit} at node {max_field.scoping.ids[0]}")
print(f"  Minimum: {min_field.data[0]:.6f} {min_field.unit} at node {min_field.scoping.ids[0]}")

# Find which time step has the overall maximum
max_values = [np.max(field.data) for field in all_disp_mag_fc]
max_time_step = np.argmax(max_values) + 1

print(f"\nOverall maximum occurs at time step: {max_time_step}")
print(f"Value: {max_values[max_time_step-1]:.6f} {all_disp_mag_fc[0].unit}")
all_disp_mag_fc[max_time_step-1].meshed_region = mesh
# Plot the time step with maximum displacement
all_disp_mag_fc[max_time_step-1].plot(
    title=f"Maximum Displacement - Time Step {max_time_step}"
)

Displacement extremes over entire analysis:
  Maximum: 0.037338 m at node 0
  Minimum: 0.000000 m at node 0

Overall maximum occurs at time step: 1
Value: 0.037338 m


Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x216ac0e2740_20&reconnect=auto" class="pyvi…

(None, <pyvista.plotting.plotter.Plotter at 0x216ac0e2740>)

## Comparison: Different Result Types

### Example 15: Plot Multiple Result Types

In [23]:
# Get different results at the same time step
time_step = [30]

# 1. Displacement magnitude
disp_op = dpf.operators.result.displacement()
disp_op.inputs.data_sources(model.metadata.data_sources)
disp_op.inputs.time_scoping(time_step)

norm_op = dpf.operators.math.norm_fc()
norm_op.inputs.fields_container(disp_op.outputs.fields_container)
disp_mag = norm_op.outputs.fields_container()[0]

# 2. Von Mises stress
stress_op = dpf.operators.result.stress()
stress_op.inputs.data_sources(model.metadata.data_sources)
stress_op.inputs.time_scoping(time_step)

vm_op = dpf.operators.invariant.von_mises_eqv_fc()
vm_op.inputs.fields_container(stress_op.outputs.fields_container)

nodal_op = dpf.operators.averaging.to_nodal_fc()
nodal_op.inputs.fields_container(vm_op.outputs.fields_container)
von_mises = nodal_op.outputs.fields_container()[0]
disp_mag.meshed_region = mesh
# Plot both
print("Plotting different result types at the same time step...\n")

disp_mag.plot(title="Displacement Magnitude")
print(f"Displacement range: {np.min(disp_mag.data):.6f} to {np.max(disp_mag.data):.6f} {disp_mag.unit}")

von_mises.plot(title="Von Mises Stress")
print(f"Stress range: {np.min(von_mises.data):.2f} to {np.max(von_mises.data):.2f} {von_mises.unit}")

Plotting different result types at the same time step...



Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x21682472c20_22&reconnect=auto" class="pyvi…

Displacement range: 0.000000 to 0.041883 m


Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x21682472680_23&reconnect=auto" class="pyvi…

Stress range: 1705526.56 to 466093820.45 Pa


## Summary of Visualization Options

### Common Plot Parameters

| Parameter | Description | Example |
|-----------|-------------|----------|
| `title` | Plot title | `title="My Plot"` |
| `deform_by` | Field to deform mesh | `deform_by=disp_field` |
| `scale_factor` | Deformation scale | `scale_factor=10.0` |
| `show_edges` | Show element edges | `show_edges=True` |
| `color` | Mesh color | `color='lightblue'` |
| `opacity` | Transparency | `opacity=0.7` |

### Visualization Workflow

1. **Extract data** using operators or model results
2. **Process data** (compute norms, invariants, averages)
3. **Plot** using `.plot()` method
4. **Customize** with parameters

### Best Practices

- Convert to **nodal location** for smoother contours
- Use appropriate **scale factors** for deformation
- Choose meaningful **color ranges** for contours
- Add **descriptive titles** to plots
- Show **deformed shape** for structural analyses
- Compare **multiple time steps** to understand evolution

## Practice Exercise

**Task**: Create a visualization workflow that:
1. Extracts Von Mises stress at time step 20
2. Converts it to nodal location
3. Gets displacement at the same time step
4. Plots the Von Mises stress on the deformed shape with scale factor 15
5. Prints the min, max, and average stress values

In [ ]:
# Your code here - implement the exercise!

# Step 1: Extract Von Mises stress

# Step 2: Convert to nodal

# Step 3: Get displacement

# Step 4: Plot on deformed shape

# Step 5: Print statistics

### Solution (Try the exercise first!)

In [24]:
# Solution to the exercise

# Step 1: Extract Von Mises stress at time step 20
stress_ex_op = dpf.operators.result.stress()
stress_ex_op.inputs.data_sources(model.metadata.data_sources)
stress_ex_op.inputs.time_scoping([20])

vm_ex_op = dpf.operators.invariant.von_mises_eqv_fc()
vm_ex_op.inputs.fields_container(stress_ex_op.outputs.fields_container)

# Step 2: Convert to nodal location
nodal_ex_op = dpf.operators.averaging.to_nodal_fc()
nodal_ex_op.inputs.fields_container(vm_ex_op.outputs.fields_container)

vm_nodal = nodal_ex_op.outputs.fields_container()[0]

# Step 3: Get displacement at time step 20
disp_ex_op = dpf.operators.result.displacement()
disp_ex_op.inputs.data_sources(model.metadata.data_sources)
disp_ex_op.inputs.time_scoping([20])

disp_ex = disp_ex_op.outputs.fields_container()[0]

# Step 4: Plot Von Mises on deformed shape
vm_nodal.plot(
    deform_by=disp_ex,
    scale_factor=15.0,
    title="Exercise: Von Mises Stress on Deformed Shape (Time Step 20)"
)

# Step 5: Print statistics
print("Exercise Solution Results:")
print("=" * 60)
print(f"Von Mises Stress Statistics at Time Step 20:")
print(f"  Minimum: {np.min(vm_nodal.data):.2f} {vm_nodal.unit}")
print(f"  Maximum: {np.max(vm_nodal.data):.2f} {vm_nodal.unit}")
print(f"  Average: {np.mean(vm_nodal.data):.2f} {vm_nodal.unit}")
print(f"  Std Dev: {np.std(vm_nodal.data):.2f} {vm_nodal.unit}")

Widget(value='<iframe src="http://localhost:49272/index.html?ui=P_0x216adcb70a0_24&reconnect=auto" class="pyvi…

Exercise Solution Results:
Von Mises Stress Statistics at Time Step 20:
  Minimum: 1758.89 Pa
  Maximum: 64448.27 Pa
  Average: 25176.35 Pa
  Std Dev: 17191.22 Pa


## Key Takeaways

✅ **Basic plotting**: Use `.plot()` on meshes and fields  
✅ **Deformed shapes**: Combine field data with displacement  
✅ **Customization**: Control appearance with parameters  
✅ **Multiple views**: Compare different time steps or result types  
✅ **Processing first**: Often need to compute norms, invariants, or convert locations before plotting  
✅ **Nodal averaging**: Creates smoother, more visually appealing contours  

Visualization is crucial for:
- Understanding simulation results
- Identifying critical regions
- Validating models
- Communicating findings

**Next Steps**: Combine operators, fields, and visualization to perform complete engineering analyses!